In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns
sns.set()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [3]:
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
client_id = "9cd3dd2ea2cf492ca28ab0247a79d781"
client_secret = "11c972ad002843e9be5ecc31f022dd6e"
grant_type = 'client_credentials'
body_params = {'grant_type' : grant_type}
url = 'https://accounts.spotify.com/api/token'

response = requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.content)

b'{"access_token":"BQAq7fJDKWG6SwYXTNBQa-74y6ZIZ_HRf7401qd-rYbex4aIGYLlBtkhvTClMymeLsBTJupMa9ceDbcgIyY","token_type":"Bearer","expires_in":3600,"scope":""}'


In [4]:
# Using Spotipy library
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

sp.audio_features(["0UaMYEvWZi0ZqiDOoHU3YI"])

[{'danceability': 0.904,
  'energy': 0.813,
  'key': 4,
  'loudness': -7.105,
  'mode': 0,
  'speechiness': 0.121,
  'acousticness': 0.0311,
  'instrumentalness': 0.00697,
  'liveness': 0.0471,
  'valence': 0.81,
  'tempo': 125.461,
  'type': 'audio_features',
  'id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
  'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI',
  'duration_ms': 226864,
  'time_signature': 4}]

In [5]:
def songs_in_playlist(df, pid):
    '''
    input: dataframe and playlist ID
    returns: list of all songs on playlist by track_uri 
    '''
    playlist = np.where(df.pid == pid)
    return list(df.iloc[playlist]["track_uri"])

def playlist_means(tracks):
    '''
    input: list of tracks on a playlist (can be generated from songs_in_playlist())
    returns: statistics for audio features of playlist  
    '''
    num = len(tracks)
    # We can only pass 100 tracks at a time
    if num <= 100:
        p = pd.DataFrame.from_records(sp.audio_features(tracks))
    else:
        start = 0
        end = 100
        p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
        for i in range(int(num / 100)):
            start = end
            diff = num - end
            if diff > 100:
                end += 100
            else:
                end += diff
            
            new_p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
            p = p.append(new_p) 
    return p

---
**Note**

We could construct a model which determines if a user will like a certain playlist based on comparing their playlists and liked playlists to the audio features of our playlist. Train model on users playlists and likes, classify playlist as 0 or 1 if the user will like the playlist.

I believe our final model will look something like this: https://developer.spotify.com/console/get-recommendations/

In [228]:
songs = pd.read_pickle("pickles/songs_30k_dropped.pkl")
songs.head()

,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0,0.03110,https://api.spotify.com/v1/audio-analysis/0UaM...,0.904,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.006970,4,0.0471,-7.105,0,0.1210,125.461,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.810
1,1,0.02490,https://api.spotify.com/v1/audio-analysis/6I9V...,0.774,198800,0.838,6I9VzXrHxO9rA9A5euc8Ak,0.025000,5,0.2420,-3.914,0,0.1140,143.040,4,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.924
2,2,0.00238,https://api.spotify.com/v1/audio-analysis/0WqI...,0.664,235933,0.758,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2,0.0598,-6.583,0,0.2100,99.259,4,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.701
3,3,0.20200,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.891,267267,0.714,1AWQoqb9bSvzTjaLralEkT,0.000234,4,0.0521,-6.055,0,0.1400,100.972,4,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.818
4,4,0.05610,https://api.spotify.com/v1/audio-analysis/1lzr...,0.853,227600,0.606,1lzr43nnXAijIGYnCT8M8H,0.000000,0,0.3130,-4.596,1,0.0713,94.759,4,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.654


In [240]:
songs2 = songs.copy()
songs2 = songs2.reset_index()
cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]

def scaler(cols, df_old, df_new):
    df_new = df_old.copy()
    x_scaler = MinMaxScaler().fit(df_old["tempo"].values.reshape(-1,1))
    x_scaled = x_scaler.transform(df_old["tempo"].values.reshape(-1,1))
    df_new["tempo"] = x_scaled

#     x_scaler.transform(df_old[col].values.reshape(-1,1))
    
#     for col in cols: 
#         x_scaler = MinMaxScaler().fit(df_old[col].values.reshape(-1,1))
#         x_scaled = MinMaxScaler().fit_transform(df_old[col].values.reshape(-1,1))
#         df_new[col] = x_scaled
        
#         x_scaler.transform(df_old[col].values.reshape(-1,1))
    return(df_new, x_scaler)


        
songs2, x_scaler = scaler(cols, songs, songs2)
songs2.head()
songs2['index']

0           0
1           1
2           2
3           3
4           4
5           5
6           6
7           7
8           8
9           9
10         10
11         11
12         12
13         13
14         14
15         15
16         16
17         17
18         18
19         19
20         20
21         21
22         22
23         23
24         24
25         25
26         26
27         27
28         28
29         29
           ..
1998486    37
1998487    38
1998488    39
1998530    81
1998531    82
1998534    85
1998535    86
1998537    88
1998538    89
1998539    90
1998540    91
1998541    92
1998542    93
1998544    95
1998545    96
1998546    97
1998547    98
1998551     2
1998555     6
1998556     7
1998558     9
1998559    10
1998560    11
1998561    12
1998646    75
1998653    82
1998664    93
1998665    94
1998700    29
1998747    16
Name: index, Length: 336251, dtype: int64

In [230]:
song1 = songs2.iloc[np.where(songs.id == '1MYYt7h6amcrauCOoso3Gx')]
song1

,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
53,1,0.000032,https://api.spotify.com/v1/audio-analysis/1MYY...,0.343,70294,0.975,1MYYt7h6amcrauCOoso3Gx,0.991,1,0.0515,-2.502,0,0.135,0.483471,4,https://api.spotify.com/v1/tracks/1MYYt7h6amcr...,audio_features,spotify:track:1MYYt7h6amcrauCOoso3Gx,0.133


In [251]:
# def distance(idkey, songs):
#     song = pd.DataFrame(sp.audio_features(idkey))
#     song_scaled = song.copy()
# #     song = scaler(cols, song, song_scaled)
#     for col in ["tempo"]:
#         song_scaled[col] = x_scaler.transform(song.loc[:, col].values.reshape(-1,1))
#         print(col, song_scaled.loc[:, col])
        
#     song = song_scaled
#     dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
#                  + (np.subtract(songs.danceability, song.danceability))**2
#                  + (np.subtract(songs.energy, song.energy))**2
#                  + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
#                  + (np.subtract(songs.liveness, song.liveness))**2
#                  + (np.subtract(songs.speechiness, song.speechiness))**2
#                  + (np.subtract(songs.tempo, song.tempo))**2)
        
#     return dist


def playlist_generator(input_song_uri, n, songDF):
    def distance(idkey, songs):
        song = pd.DataFrame(sp.audio_features(idkey))
        song.loc[:,"tempo"] = x_scaler.transform(song.loc[:,"tempo"].values.reshape(-1,1))
        dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
                     + (np.subtract(songs.danceability, song.danceability))**2
                     + (np.subtract(songs.energy, song.energy))**2
                     + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
                     + (np.subtract(songs.liveness, song.liveness))**2
                     + (np.subtract(songs.speechiness, song.speechiness))**2
                     + (np.subtract(songs.tempo, song.tempo))**2)

        return dist
    
    topn_dist = distance(input_song_uri, songDF).sort_values()[0:n]
    print(topn_dist.index)
    topn = songs2.loc[topn_dist.index]
    return topn

def playlist_printer(topn_ids):
    for ref in topn_ids['id']:
        track = sp.track(ref)
        print(track['name'])
        print('by')
        for artist in track['artists']:
            print(artist['name'])
        print('-----------------------------------')
        

topn_ids = playlist_generator('5cYA45RVGI6F4f06gtWjsd', 25, songs2)
playlist_printer(topn_ids)

Int64Index([  55928, 1267997,  493042,  731727,  424604,  214591,  425484,
            1149192,  716407, 1708337,  731720, 1814775,  558433, 1161021,
             964127,  731670,  309518, 1977752, 1010140, 1445269,  554046,
            1327074, 1049272,  796622,  579614],
           dtype='int64')
The Ribbon
by
Rodrigo Amarante
-----------------------------------
Saint-Saëns: Introduction et rondo capriccioso, Op. 28
by
Camille Saint-Saëns
Itzhak Perlman
Jean Martinon
Orchestre de Paris
-----------------------------------
Lake in Autumn - Dream Version
by
Drew Cox
-----------------------------------
Search for Love
by
Martin Sasse Trio
Peter Bernstein
-----------------------------------
Latika's Theme (From "Slumdog Millionaire")
by
London Music Works
-----------------------------------
Rain
by
Brian Crain
-----------------------------------
Not So Fireproof
by
John Powell
-----------------------------------
Beautiful Birds - Instrumental
by
Passenger
---------------------------------

In [206]:
top10_dist = distance('5IVuqXILoxVWvWEPm82Jxr', songs2).sort_values()[0:10]

tempo 0    0.399919
Name: tempo, dtype: float64


In [207]:
top10_dist.index

Int64Index([2, 58630, 186149, 260470, 168051, 175174, 231675, 196323, 184675,
            57254],
           dtype='int64')

In [208]:
top10 = songs2.iloc[top10_dist.index]

In [209]:
hrefs = top10['id']
hrefs

2         0WqIKmW4BTrj3eJFmnCKMv
58630     25jgrjoFiABoU8yV9IKG3a
186149    0Jwqf8WzYlEtywz7opb9Uk
260470    4YHXwcGElKDvJpzRHf5LYj
168051    6lbzLUoma85KVbQqBpvBrw
175174    3fYsPWgd6ZbwWsroDsVaSS
231675    7srHFjnIelAi3SCYnNJ1VL
196323    29eTXdNGfTWDy2Q6Eg8zhZ
184675    6s4AHi9vOHiKwlu5vZwik5
57254     2bue03iRcaiRGEvhn0FU8K
Name: id, dtype: object

In [210]:
for ref in top10['id']:
    track = sp.track(ref)
    print(track['name'])
    print('by')
    for artist in track['artists']:
        print(artist['name'])
    print('-----------------------------------')

Crazy In Love (feat. Jay-Z)
by
Beyoncé
JAY Z
-----------------------------------
Stand A Chance
by
Damian Marley
Treach
Yami Bolo
-----------------------------------
Welcome To LA
by
Oliver Tree
-----------------------------------
Never Gonna Let You Go (She's a Keepa) - No Rap Version
by
Omarion
-----------------------------------
The Doo-Bop Song
by
Miles Davis
-----------------------------------
Red Bull & Vodka
by
Sunny Ledfurd
-----------------------------------
Inferno
by
Empire Cast
Remy Ma
Sticky Fingaz
-----------------------------------
OMG
by
RJMrLA
-----------------------------------
Dance to It
by
Tut Tut Child
-----------------------------------
Thank You
by
Busta Rhymes
Q-Tip
Kanye West
Lil Wayne
-----------------------------------
